In [ ]:
import os
import ee
import geemap
import math
Map = geemap.Map()
ee.Initialize()
Map

In [ ]:
import urllib.request

In [ ]:
geemap.ee_search()

In [ ]:
def generateGrid(xmin, xmax, dx, y1, y2):
    xx = ee.List.sequence(xmin, ee.Number(xmax).subtract(dx), dx)
    cells = xx.map(func_uke).flatten()
    return ee.FeatureCollection(cells)

def func_uke(x):
    x1 = ee.Number(x)
    x2 = ee.Number(x).add(ee.Number(dx))
    coords = ee.List([x1, y1, x2, y2])
    rect = ee.Algorithms.GeometryConstructors.Rectangle(coords)
    return ee.Feature(rect)

In [ ]:
def func_feh(feature):                 
    EvergreenNeedleleafForests = feature.getNumber('EvergreenNeedleleafForests')
    EvergreenBroadleafForests = feature.getNumber('EvergreenBroadleafForests')
    DeciduousNeedleleafForests = feature.getNumber('DeciduousNeedleleafForests')
    DeciduousBroadleafForests = feature.getNumber('DeciduousBroadleafForests')
    MixedForests = feature.getNumber('MixedForests')
    ClosedShrublands = feature.getNumber('ClosedShrublands')
    OpenShrublands = feature.getNumber('OpenShrublands')
    WoodySavannas = feature.getNumber('WoodySavannas')
    Savannas = feature.getNumber('Savannas')
    Grasslands = feature.getNumber('Grasslands')
    PermanentWetlands = feature.getNumber('PermanentWetlands')
    Croplands = feature.getNumber('Croplands')
    UrbanBuiltupLands = feature.getNumber('UrbanBuiltupLands')
    CroplandNaturalVegetationMosaics = feature.getNumber('CroplandNaturalVegetationMosaics')
    PermanentSnowandIce = feature.getNumber('PermanentSnowandIce')
    Barren = feature.getNumber('Barren')
    WaterBodies = feature.getNumber('WaterBodies')
    Unclassified = feature.getNumber('Unclassified')
    Land = feature.getNumber('Land')
    
    return feature.set({
        'EvergreenNeedleleafForests':EvergreenNeedleleafForests.divide(Land),
        'EvergreenBroadleafForests':EvergreenBroadleafForests.divide(Land),
        'DeciduousNeedleleafForests':DeciduousNeedleleafForests.divide(Land),
        'DeciduousBroadleafForests':DeciduousBroadleafForests.divide(Land),
        'MixedForests':MixedForests.divide(Land),
        'ClosedShrublands':ClosedShrublands.divide(Land),
        'OpenShrublands':OpenShrublands.divide(Land),
        'WoodySavannas':WoodySavannas.divide(Land),
        'Savannas':Savannas.divide(Land),
        'Grasslands':Grasslands.divide(Land),
        'PermanentWetlands':PermanentWetlands.divide(Land),
        'Croplands':Croplands.divide(Land),
        'UrbanBuiltupLands':UrbanBuiltupLands.divide(Land),
        'CroplandNaturalVegetationMosaics':CroplandNaturalVegetationMosaics.divide(Land),
        'PermanentSnowandIce':PermanentSnowandIce.divide(Land),
        'Barren':Barren.divide(Land),
        'WaterBodies':WaterBodies.divide(Land),
        'Unclassified':Unclassified.divide(Land),
        'Land':Land.divide(Land)

     })

In [ ]:
import requests
from retrying import retry
@retry(stop_max_attempt_number=1000, wait_fixed=2000)
def get_request(url):
    response = urllib.request.urlretrieve(url, filename=out_ndvi_stats)
    return response

In [ ]:
dx = 1/120
xmin = -180
xmax = 180
ymin = -65
ymax = 90
yrange = round((ymax - ymin  - 0.0001)/dx)+1
print(yrange)

In [ ]:
for year_index in range(0,21):
    year = 2000 + year_index
    year_begin = ee.Date.fromYMD(year,1,1)
    year_end = ee.Date.fromYMD(year,12,31)
    image_lc = ee.ImageCollection('MODIS/006/MCD12Q1').filterDate(year_begin , year_end).select('LC_Type1').mean()
    EvergreenNeedleleafForests = image_lc.eq(1)
    EvergreenBroadleafForests = image_lc.eq(2)
    DeciduousNeedleleafForests = image_lc.eq(3)
    DeciduousBroadleafForests = image_lc.eq(4)
    MixedForests = image_lc.eq(5)
    ClosedShrublands = image_lc.eq(6)
    OpenShrublands = image_lc.eq(7)
    WoodySavannas = image_lc.eq(8)
    Savannas = image_lc.eq(9)
    Grasslands = image_lc.eq(10)
    PermanentWetlands = image_lc.eq(11)
    Croplands = image_lc.eq(12)
    UrbanBuiltupLands = image_lc.eq(13)
    CroplandNaturalVegetationMosaics = image_lc.eq(14)
    PermanentSnowandIce = image_lc.eq(15)
    Barren = image_lc.eq(16)
    WaterBodies = image_lc.eq(17)
    Unclassified = image_lc.eq(255)

    Land = image_lc.lte(255)
    cond = ee.Image.cat(
    EvergreenNeedleleafForests.eq(1).rename('EvergreenNeedleleafForests'),
    EvergreenBroadleafForests.eq(1).rename('EvergreenBroadleafForests'),
    DeciduousNeedleleafForests.eq(1).rename('DeciduousNeedleleafForests'),
    DeciduousBroadleafForests.eq(1).rename('DeciduousBroadleafForests'),
    MixedForests.eq(1).rename('MixedForests'),
    ClosedShrublands.eq(1).rename('ClosedShrublands'),
    OpenShrublands.eq(1).rename('OpenShrublands'),
    WoodySavannas.eq(1).rename('WoodySavannas'),
    Savannas.eq(1).rename('Savannas'),
    Grasslands.eq(1).rename('Grasslands'),
    PermanentWetlands.eq(1).rename('PermanentWetlands'),
    Croplands.eq(1).rename('Croplands'),
    UrbanBuiltupLands.eq(1).rename('UrbanBuiltupLands'),
    CroplandNaturalVegetationMosaics.eq(1).rename('CroplandNaturalVegetationMosaics'),
    PermanentSnowandIce.eq(1).rename('PermanentSnowandIce'),
    Barren.eq(1).rename('Barren'),
    WaterBodies.eq(1).rename('WaterBodies'),
    Unclassified.eq(1).rename('Unclassified'),
    Land.eq(1).rename('Land'))

    for i in range (0,yrange):
        try:
            y1 = ymin + i*dx
            y2 = ymin + (i+1)*dx
            grid = generateGrid(xmin, xmax, dx, y1, y2)
            npsRes = cond.reduceRegions(grid, ee.Reducer.sum(), 500)
            npsRes = npsRes.map(func_feh)
            name='GLC_FCS30_'+'China_Fishnet_'+str(i)+'.csv'
            out_dir = 'G:\\LUCC_China\\MCD12Q1_LC\\CN_Fishnet_'+str(year)
            out_ndvi_stats = os.path.join(out_dir, name)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
            url = npsRes.getDownloadURL(filetype="csv", selectors=['TARGET_FID', 'EvergreenNeedleleafForests',
                                                                    'EvergreenBroadleafForests',
                                                                    'DeciduousNeedleleafForests',
                                                                    'DeciduousBroadleafForests',
                                                                    'MixedForests',
                                                                    'ClosedShrublands',
                                                                    'OpenShrublands',
                                                                    'WoodySavannas',
                                                                    'Savannas',
                                                                    'Grasslands',
                                                                    'PermanentWetlands',
                                                                    'Croplands',
                                                                    'UrbanBuiltupLands',
                                                                    'CroplandNaturalVegetationMosaics',
                                                                    'PermanentSnowandIce',
                                                                    'Barren',
                                                                    'WaterBodies',
                                                                    'Unclassified',
                                                                    'Land'], filename=name)

            print(url)
            print(out_ndvi_stats)
            get_request(url)
        except Exception as e:
            print(e)       